In [ ]:
!wget test.zip 'https://downloader.disk.yandex.ru/disk/5082a607dea0aacc3d20e5b441cd4b7533da5a650df3f567501ad8a154f763e4/6851963e/hHsHnWApe_ASqMVFpLHBF6_9rCBsLrgaKF5kp2qzYrG5Krh0oqrxe_ldyi5jK0idd-DFUHk89N_oBbBMveda1Q%3D%3D?uid=0&filename=data_test_short.zip&disposition=attachment&hash=PnPZwPXAqeDhKB3yN%2B/iyx%2B0J6ojxMo4GIRK203mwii3yV5av71luXdLBW5PxsiRq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fzip&owner_uid=990870740&fsize=12895585234&hid=bd7c4ac68533d5552604210731df433a&media_type=compressed&tknv=v3'
!wget train.zip 'https://downloader.disk.yandex.ru/disk/2d2c9b68960de598f5eb03661a2fbb181d9d80a8f3a20d9472af2934e1b0050e/68519530/hHsHnWApe_ASqMVFpLHBF8Unvfzm5Nl_X4JrfVLIuvqJzvXq6WpV4kEFZ7076ChtuPRZ3YnMw52g7A5L4wtjpg%3D%3D?uid=0&filename=data_train_short.zip&disposition=attachment&hash=Nf16sYxqnky4DGlWR6KBD6MM6H0CK5TuX7lvl6JH0h/k0fC5z12y6wx8F8R/kT2Sq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fzip&owner_uid=990870740&fsize=19987339290&hid=5979f7f50fc5e10e0c0c51358ee33d88&media_type=compressed&tknv=v3'

--2025-06-17 12:28:18--  http://test.zip/
Resolving test.zip (test.zip)... failed: Name or service not known.
wget: unable to resolve host address ‘test.zip’
The destination name is too long (388), reducing to 236
--2025-06-17 12:28:18--  https://downloader.disk.yandex.ru/disk/5082a607dea0aacc3d20e5b441cd4b7533da5a650df3f567501ad8a154f763e4/6851963e/hHsHnWApe_ASqMVFpLHBF6_9rCBsLrgaKF5kp2qzYrG5Krh0oqrxe_ldyi5jK0idd-DFUHk89N_oBbBMveda1Q%3D%3D?uid=0&filename=data_test_short.zip&disposition=attachment&hash=PnPZwPXAqeDhKB3yN%2B/iyx%2B0J6ojxMo4GIRK203mwii3yV5av71luXdLBW5PxsiRq/J6bpmRyOJonT3VoXnDag%3D%3D&limit=0&content_type=application%2Fzip&owner_uid=990870740&fsize=12895585234&hid=bd7c4ac68533d5552604210731df433a&media_type=compressed&tknv=v3
Resolving downloader.disk.yandex.ru (downloader.disk.yandex.ru)... 77.88.21.127, 2a02:6b8::2:127
Connecting to downloader.disk.yandex.ru (downloader.disk.yandex.ru)|77.88.21.127|:443... connected.
HTTP request sent, awaiting response... 302 Found
Loca

In [ ]:
import os
os.makedirs('/content/train', exist_ok=True)
os.makedirs('/content/test', exist_ok=True)
!unzip /content/train.zip -d /content/train
!unzip /content/test.zip -d /content/test

Archive:  /content/train.zip
   creating: /content/train/data_train_short/
   creating: /content/train/data_train_short/-220020068_456255414/
  inflating: /content/train/data_train_short/-220020068_456255414/-220020068_456255414.mp4  
   creating: /content/train/data_train_short/-220020068_456249693/
  inflating: /content/train/data_train_short/-220020068_456249693/-220020068_456249693.mp4  
   creating: /content/train/data_train_short/-220020068_456255339/
  inflating: /content/train/data_train_short/-220020068_456255339/-220020068_456255339.mp4  
   creating: /content/train/data_train_short/-220020068_456241755/
  inflating: /content/train/data_train_short/-220020068_456241755/-220020068_456241755.mp4  
   creating: /content/train/data_train_short/-220020068_456241671/
  inflating: /content/train/data_train_short/-220020068_456241671/-220020068_456241671.mp4  
   creating: /content/train/data_train_short/-220020068_456255340/
  inflating: /content/train/data_train_short/-220020068_45

In [ ]:
import os
import json
import cv2
import torch
import numpy as np
from torchvision import models, transforms
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

def extract_frame_features(video_path, model, transform, fps=1, max_seconds=None):
    cap = cv2.VideoCapture(video_path)
    video_fps = cap.get(cv2.CAP_PROP_FPS)
    step = max(1, int(video_fps // fps))
    features = []
    idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        current_time = idx / video_fps
        if max_seconds and current_time > max_seconds:
            break
        if idx % step == 0:
            img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (256, 256))
            img = transform(img).unsqueeze(0).to(device)
            with torch.no_grad():
                feat = model(img).cpu().numpy().flatten()
            features.append(feat)
        idx += 1

    cap.release()
    if features:
        return np.vstack(features)
    else:
        return np.empty((0, feat_dim))
def get_intro_template(train_dir, labels_json, model, transform, fps=1):
    with open(labels_json) as f:
        labels = json.load(f)
    all_feats = []
    end_times = []
    for vid_id, meta in tqdm(labels.items()):
        start_s = sum(int(x) * 60 ** i for i, x in enumerate(reversed(meta['start'].split(':'))))
        end_s = sum(int(x) * 60 ** i for i, x in enumerate(reversed(meta['end'].split(':'))))
        end_times.append(end_s)
        video_path = os.path.join(train_dir, vid_id, vid_id + '.mp4')
        feats = extract_frame_features(video_path, model, transform, fps)
        idx_start = int(start_s * fps)
        idx_end = min(int(end_s * fps), feats.shape[0])
        all_feats.append(feats[idx_start:idx_end])
    max_end = max(end_times)
    lengths = [f.shape[0] for f in all_feats if f.shape[0] > 0]
    L = int(np.median(lengths))
    import cv2 as cv
    resized = []
    for f in all_feats:
        if f.shape[0] == 0:
            continue
        resized.append(cv.resize(f, (f.shape[1], L)).T)
    template = np.mean(np.stack(resized, axis=0), axis=0).T
    return template, max_end

def find_intro_in_video(video_path, template, model, transform, max_seconds, fps=1):
    feats = extract_frame_features(video_path, model, transform, fps, max_seconds)
    L = template.shape[0]
    best = (-1, 0)
    for start in range(0, feats.shape[0] - L + 1):
        window = feats[start:start+L]
        sim = cosine_similarity(window.mean(axis=0, keepdims=True), template.mean(axis=0, keepdims=True))[0,0]
        if sim > best[0]:
            best = (sim, start)
    _, best_start = best
    start_s = best_start / fps
    end_s = (best_start + L) / fps
    return start_s, end_s

if __name__ == '__main__':
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    cnn = models.resnet18(pretrained=True)
    cnn.fc = torch.nn.Identity()
    cnn = cnn.to(device).eval()
    dummy = torch.zeros(1,3,256,256).to(device)
    feat_dim = cnn(dummy).shape[1]
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    train_data_path = '/content/train/data_train_short'
    train_labels_path = os.path.join(train_data_path, 'labels.json')
    test_data_path = '/content/test/data_test_short'

    template, max_end = get_intro_template(train_data_path, train_labels_path, cnn, transform, fps=1)

    results = {}
    for vid_id in tqdm(os.listdir(test_data_path), desc='Processing test videos'):
        path = os.path.join(test_data_path, vid_id, vid_id + '.mp4')
        start_s, end_s = find_intro_in_video(path, template, cnn, transform, max_end, fps=1)
        results[vid_id] = {
            'start': f"0:00:{int(start_s):02d}",
            'end': f"0:00:{int(end_s):02d}"
        }
    with open('predicted_labels.json', 'w') as f:
        json.dump(results, f, indent=2)
    print("Done: predicted_labels.json saved.")


In [40]:
import json

def parse_pred_time(t):

    h, m, last = t.split(':')
    base = int(h)*3600 + int(m)*60
    last_i = int(last)
    if last_i > 59:
        return base + last_i / 1000.0
    else:
        #
        return base + last_i

def parse_true_time(t):
    h, m, s = t.split(':')
    return int(h)*3600 + int(m)*60 + int(s)

with open('predicted_labels.json', 'r', encoding='utf-8') as f:
    pred = json.load(f)

with open('test/data_test_short/labels.json', 'r', encoding='utf-8') as f:
    true = json.load(f)

ious = {}
for vid, p in pred.items():
    if vid not in true:
        continue
    ps = parse_pred_time(p['start'])
    pe = parse_pred_time(p['end'])
    ts = parse_true_time(true[vid]['start'])
    te = parse_true_time(true[vid]['end'])

    inter_start = max(ps, ts)
    inter_end   = min(pe, te)
    intersection = max(0.0, inter_end - inter_start)

    union = (pe - ps) + (te - ts) - intersection
    iou = intersection / union if union > 0 else 0.0
    ious[vid] = {
        'pred_interval': (ps, pe),
        'true_interval': (ts, te),
        'intersection_s': intersection,
        'union_s': union,
        'iou': iou
    }

for vid, stats in ious.items():
    print(f"Видео {vid}:")
    print(f"  Predicted: {stats['pred_interval'][0]:.3f}s – {stats['pred_interval'][1]:.3f}s")
    print(f"  Ground‑truth: {stats['true_interval'][0]:.3f}s – {stats['true_interval'][1]:.3f}s")
    print(f"  Intersection: {stats['intersection_s']:.3f}s")
    print(f"  Union:        {stats['union_s']:.3f}s")
    print(f"  IoU:          {stats['iou']:.3f}")
    print()

  =


Видео -220020068_456255411:
  Predicted: 0.529s – 0.534s
  Ground‑truth: 102.000s – 107.000s
  Intersection: 0.000s
  Union:        5.005s
  IoU:          0.000

Видео -220020068_456249313:
  Predicted: 0.341s – 0.346s
  Ground‑truth: 5.000s – 20.000s
  Intersection: 0.000s
  Union:        15.005s
  IoU:          0.000

Видео -220020068_456249344:
  Predicted: 20.000s – 25.000s
  Ground‑truth: 5.000s – 16.000s
  Intersection: 0.000s
  Union:        16.000s
  IoU:          0.000

Видео -220020068_456249272:
  Predicted: 19.000s – 24.000s
  Ground‑truth: 5.000s – 15.000s
  Intersection: 0.000s
  Union:        15.000s
  IoU:          0.000

Видео -220020068_456249275:
  Predicted: 0.223s – 0.228s
  Ground‑truth: 5.000s – 15.000s
  Intersection: 0.000s
  Union:        10.005s
  IoU:          0.000

Видео -220020068_456249358:
  Predicted: 0.210s – 0.215s
  Ground‑truth: 5.000s – 15.000s
  Intersection: 0.000s
  Union:        10.005s
  IoU:          0.000

Видео -220020068_456249716:
  Pred

#**Sadly, I did not have enough time to optimize my solution and gain better results. The solution represented here was done in one try, and I might've done this better if I had a chance to try more things. Here are my assumptions to make it better in some further, hypothetical work::**
1. My current method of averaging all the features in a window and then comparing that single average vector is just too basic.  I would replace the cosine_similarity on the mean vectors with Dynamic Time Warping (DTW).
2.  Instead of one master template, I would treat my collection of training intros as a gallery of examples. When scanning a test video, I'd use DTW to compare the current window against every single intro from the training set.
3. I would use a library like librosa to extract audio features from the videos. I'd start with something standard like MFCCs (Mel-Frequency Cepstral Coefficients) for every second of audio. Then, I could either search for the audio pattern independently and combine the scores, or even concatenate the audio feature vector with my visual feature vector for each time step.
4. I would increase the frame rate to at least 5 FPS to capture more temporal detail. As a more advanced step, if I still wasn't getting the accuracy I wanted, I would explore feeding my sequence of ResNet features into a simple LSTM (Long Short-Term Memory) network. This would help the model learn the expected patterns of change between frames, getting closer to true video understanding rather than just image matching.